`pidgin.display` is a collection of display rules for IPython objects.  

`StringFormatter` is a class that adds reules for string `object`s.

In [1]:
    import abc, fnmatch, io, dataclasses, functools, IPython, jinja2, inspect, collections, pygments, mistune, inspect, importlib, types, base64, string, traitlets
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    with __import__('importnb').Notebook():
        try: 
            from .utils import format_images, finalize_ipython
        except: 
            from utils import format_images, finalize_ipython
    ip = IPython.get_ipython()

In [2]:
def strip_graphviz_headers(Source):
    str = Source._repr_svg_()
    # Mistune does a bad job of handling inline comments.
    for i in range(4): str = str.partition('>')[2]

    # Mistune has a hard time with the xmlns information.
    for year in (1999, 2000): str = str.replace(F'xmlns:xlink="http://www.w3.org/{year}/xlink"', '')
    return str.lstrip()

In [3]:
def graphviz(ip, str): 
    try: import graphviz
    except: return
    if '->' in str and not str.startswith('di'): str = 'di' + str
    return IPython.display.HTML(strip_graphviz_headers(graphviz.Source(str)))

In [4]:
def eval_shorthand_ipython(ip, str):
    return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [5]:
def flatten(str): return ''.join(str.splitlines())

In [6]:
def embed(ip, str):
    type = guess(str) or ''
    if str.startswith('http'): 
        if type.startswith('image') and not type.endswith('svg'):
            return IPython.display.Image(str)
        return IPython.display.IFrame(str, '100%', '500px')


In [7]:
    @dataclasses.dataclass(order=True)
    class PidginDisplay:
        data: str = ''
        environment: jinja2.Environment = dataclasses.field(
            default_factory=jinja2.Environment
        )
        
        ns: dict = dataclasses.field(default_factory=dict, repr=False)

        def render(self):
            return self.environment.from_string(self.format()).render(**(self.ns))
        
        def format(self, **dict): 
            return mistune.markdown(self.data, 
                                    escape=False, 
                                    parse_block_html=True, 
                                    parse_inline_html=True, **dict)
        
        def _repr_mimebundle_(self, include=None, exclude=None):
            import builtins
            self.environment.globals.update(vars(builtins))
            return {
                'text/html': self.render(), 
                'text/markdown': self.data, 'text/plain': self.data}, {}

    IPython.core.formatters.MimeBundleFormatter.for_type_by_name

In [8]:
environment = jinja2.Environment(finalize=finalize_ipython, lstrip_blocks=True)

In [9]:
class StringFormatterBase(abc.ABCMeta):
    level = 0
    environment = environment
    def __enter__(cls): cls.level += 1
    def __exit__(cls, *exc): cls.level -= 1
        
    def format(cls, object, include=None, exclude=None, display=False):
        with cls:
            for type, callable in cls.formatters.items():
                if fnmatch.fnmatch(object, type):
                    result = callable(ip, object)
                    if result: return ip.display_formatter.format(result)
            if cls.level == 1: 
                return ip.display_formatter.format(PidginDisplay(object, cls.environment, ip.user_ns))

In [10]:
class StringFormatter(metaclass=StringFormatterBase):
    formatters = {'https://*' : embed,
                  'http://*' : embed,
                  'graph {*}': graphviz, 
                  'digraph {*}': graphviz,
                  ';[! ]*': eval_shorthand_ipython,
                  ',[! ]*': eval_shorthand_ipython,}

In [11]:
def format_doc(object, **dict):
    if object.__doc__:
        if inspect.isfunction(object) or inspect.isclass(object): 
            name = object.__module__
        if inspect.ismodule(object):
            name = object.__name__
        with StringFormatter:
            # Context fixes weird indents.
            return ip.display_formatter.format(
                PidginDisplay(object.__doc__, environment=environment, ns=vars(importlib.import_module(name))))

In [12]:
   def show_axes(object):
        ip = IPython.get_ipython()
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(object.figure).print_png(bytes)
        bundle = {
            'image/png': bytes.getvalue()
        }
        return {
            'text/html': format_images('image/png', bundle),
            'text/plain': repr(object),
        }, {}

In [13]:
    def format_frame(DataFrame):
        return {
            'text/html': F"<span>{DataFrame._repr_html_().lstrip('<div>').rstrip('</div>')}</span>",
            'text/plain': repr(DataFrame)
        }, {}

In [14]:
def load_ipython_extension(ip): 
    ip = ip or IPython.get_ipython()
    ip.display_formatter.mimebundle_formatter.for_type(type, format_doc)
    ip.display_formatter.mimebundle_formatter.for_type_by_name('matplotlib.figure', 'Axes', show_axes)  
    ip.display_formatter.mimebundle_formatter.for_type_by_name('matplotlib.axes._subplots', 'AxesSubplot', show_axes)    
    ip.display_formatter.mimebundle_formatter.for_type_by_name('pandas.core.frame', 'DataFrame', format_frame)    
    ip.display_formatter.mimebundle_formatter.for_type(types.ModuleType, format_doc)
    ip.display_formatter.mimebundle_formatter.for_type(str, StringFormatter.format)
    
def unload_ipython_extension(ip=None): 
    for object in (type, types.ModuleType, str):
        ip.display_formatter.mimebundle_formatter.type_printers.pop(object)
    for object in (('matplotlib.figure', 'Axes'), ('pandas.core.frame', 'DataFrame')):
        ip.display_formatter.mimebundle_formatter.deferred_printers.pop(object)
    

__name__ == '__main__' and load_ipython_extension(get_ipython())